# Fine tuning, for the n=4 case

In [1]:
import torch
import torch.nn as nn

# Needed for parallel 
from collections import OrderedDict

# For training 
from network_architecture_v2 import MyBertForSequenceClassification

# For fine tuning
from datasets import load_dataset #, load_metric
from transformers import BertTokenizer
from transformers import Trainer, TrainingArguments
import numpy as np

In [2]:
# Load dataset
dataset = load_dataset('glue', 'sst2')

# I believe this is the tokenizer I used... 
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples['sentence'], padding="max_length", 
                     max_length=128, truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

/home/sjiang/braids/pip-test/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


# Load the parallel model

This involves a bit more code

In [3]:
checkpoint_0 = torch.load('bert-save-4/model_checkpoint_0_batch_idx=80000')
checkpoint_1 = torch.load('bert-save-4/model_checkpoint_1_batch_idx=80000')
checkpoint_2 = torch.load('bert-save-4/model_checkpoint_2_batch_idx=80000')
checkpoint_3 = torch.load('bert-save-4/model_checkpoint_3_batch_idx=80000')

In [4]:
keys_0 = checkpoint_0['model_state'].keys()
keys_1 = checkpoint_1['model_state'].keys()
keys_2 = checkpoint_2['model_state'].keys()
keys_3 = checkpoint_3['model_state'].keys()

In [11]:
# Ugh, this is so dumb
new_dict = OrderedDict()
keys_0 = checkpoint_0['model_state'].keys()
counter = 0
for key in keys_0:
    if 'parallel_nn' in key:
        split = key.split('.')
        split[1] = 'serial_nn'
        if int(split[2]) > counter:
            counter = int(split[2])
            
        split.insert(3, 'layer')
        new_key = '.'.join(split[1:])
        new_dict[new_key] = checkpoint_0['model_state'][key]
    else:
        new_key = key
        if 'close_nsp' in key:
            # print(key)
            split = key.split('.')
            split[0] = 'close_nn_nsp'
            new_key = '.'.join(split)
        if 'close_mlm' in key:
            # print(key)
            split = key.split('.')
            split[0] = 'close_nn_mlm'
            new_key = '.'.join(split)
        
        new_dict[new_key] = checkpoint_0['model_state'][key]
        
print(counter)

# Now for the remaining parts? 
keys_1 = checkpoint_1['model_state'].keys()
new_counter = 0
for key in keys_1:
    if 'parallel_nn' in key:
        split = key.split('.')
        split[1] = 'serial_nn'
        split[2] = str(int(split[2]) + counter + 1)
        split.insert(3, 'layer')

        if int(split[2]) > new_counter:
            new_counter = int(split[2])
        new_key = '.'.join(split[1:])
        # print(key, new_key)
        new_dict[new_key] = checkpoint_1['model_state'][key]
    else:
        new_dict[key] = checkpoint_1['model_state'][key]

print(new_counter)
counter = new_counter
new_counter = 0
keys_2 = checkpoint_2['model_state'].keys()
for key in keys_2:
    if 'parallel_nn' in key:
        split = key.split('.')
        split[1] = 'serial_nn'
        split[2] = str(int(split[2]) + counter + 1)
        split.insert(3, 'layer')
        if int(split[2]) > new_counter:
            new_counter = int(split[2])
        new_key = '.'.join(split[1:])
        # print(key, new_key)
        new_dict[new_key] = checkpoint_2['model_state'][key]
    else:
        new_dict[key] = checkpoint_2['model_state'][key]

print(new_counter)
counter = new_counter
new_counter = 0
keys_3 = checkpoint_3['model_state'].keys()
for key in keys_3:
    if 'parallel_nn' in key:
        split = key.split('.')
        split[1] = 'serial_nn'
        split[2] = str(int(split[2]) + counter + 1)
        split.insert(3, 'layer')
        if int(split[2]) > new_counter:
            new_counter = int(split[2])
        new_key = '.'.join(split[1:])
        # print(key, new_key)
        new_dict[new_key] = checkpoint_3['model_state'][key]
    else:
        new_dict[key] = checkpoint_3['model_state'][key]

16
32
48


In [12]:
model_parallel = torch.load('serialnet_bert_32')
model_parallel.load_state_dict(new_dict)

RuntimeError: Error(s) in loading state_dict for SerialNet:
	Unexpected key(s) in state_dict: "serial_nn.32.layer.layer.layer.attention.self.query.weight", "serial_nn.32.layer.layer.layer.attention.self.query.bias", "serial_nn.32.layer.layer.layer.attention.self.key.weight", "serial_nn.32.layer.layer.layer.attention.self.key.bias", "serial_nn.32.layer.layer.layer.attention.self.value.weight", "serial_nn.32.layer.layer.layer.attention.self.value.bias", "serial_nn.32.layer.layer.layer.attention.self.dropout.p", "serial_nn.32.layer.layer.layer.attention.self.dropout.t", "serial_nn.32.layer.layer.layer.attention.LayerNorm.weight", "serial_nn.32.layer.layer.layer.attention.LayerNorm.bias", "serial_nn.32.layer.layer.layer.attention.dense.weight", "serial_nn.32.layer.layer.layer.attention.dense.bias", "serial_nn.32.layer.layer.layer.attention.dropout.p", "serial_nn.32.layer.layer.layer.attention.dropout.t", "serial_nn.32.layer.layer.layer.ff.dense1.weight", "serial_nn.32.layer.layer.layer.ff.

In [ ]:
training_parallel = MyBertForSequenceClassification(model_parallel)

# With weights loaded, go ahead and train

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    learning_rate=1e-4,
    adam_beta1=0.9,
    adam_beta2=0.988,
    adam_epsilon=1e-6,
    dataloader_drop_last=True,
    warmup_steps=100,
    weight_decay=1e-4,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
)


In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = (predictions == labels).astype(np.float32).mean().item()
    return {"accuracy": accuracy}

In [ ]:
# Initialize the Trainer
trainer = Trainer(
    model=training_parallel,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    compute_metrics=compute_metrics
)


In [ ]:
trainer.train()

In [ ]:
sum(p.numel() for p in training_parallel.parameters() if p.requires_grad)